# Oil spill data

Downloaded from the [Railroad commission](http://www.rrc.state.tx.us/oil-gas/compliance-enforcement/h-8).

The H-8 Loss Reports for crude oil, gas well liquids or associated products are available for download as an Excel compressed zip file by year. If you have questions or need assistance, contact Field Operations at 512-463-6830 or OGFOPS@rrc.texas.gov.


Looking at these files to compare the Magellan Midstream Partners spill of July 13 in Bastrop County. It's been reported to be 87,528 gallons or is 2,084 barrel. RRC tells Andy Sevilla the data uses barrels.

#### Our coverage
- http://www.statesman.com/news/local/new-data-shows-bastrop-county-oil-spill-leaked-000-gallons/Y8YK4xM4uPE37actcJi1HP/
- http://www.statesman.com/news/local/rolled-over-wheeler-spills-oil-onto-bastrop-roadway/PMhcfaqCXQ9U4V6FCX36WL/
- http://www.mystatesman.com/news/opinion/commentary-what-the-bastrop-oil-spill-says-about-effective-regulation/8wkZRUj2ts2mXR8BZhDkQM/


## Data prep

Doing the import, setting data types, column names and stacking.

In [54]:
import agate
import agateexcel
import re
import warnings
warnings.filterwarnings('ignore')

In [55]:
#bastrop data
bastrop_barrels = 2084
bastrop_gallons = '87528'

In [56]:
# sets column types where needed
specified_type = {
    'MasterYear': agate.Text(),
    'Water': agate.Text(),
    'WaterName': agate.Text()
}
# importing each file as a table
# should be refactored as a loop
h82009 = agate.Table.from_xlsx('raw-data/h8s-2009.xlsx', column_types=specified_type)
h82010 = agate.Table.from_xlsx('raw-data/h8s-2010.xlsx', column_types=specified_type)
h82011 = agate.Table.from_xlsx('raw-data/h8s-2011.xlsx', column_types=specified_type)
h82012 = agate.Table.from_xlsx('raw-data/h8s-2012.xlsx', column_types=specified_type)
h82013 = agate.Table.from_xlsx('raw-data/h8s-2013.xlsx', column_types=specified_type)
h82014 = agate.Table.from_xlsx('raw-data/h8s-2014.xlsx', column_types=specified_type)
h82015 = agate.Table.from_xlsx('raw-data/h8s-2015.xlsx', column_types=specified_type)
h82016 = agate.Table.from_xlsx('raw-data/h8s-2016.xlsx', column_types=specified_type)
h82017 = agate.Table.from_xlsx('raw-data/h8s-2017.xlsx', column_types=specified_type)

In [57]:
# renaming columns in some files for consistency.
# this could/should be refacted as a loop

column_names = {
    'Form Number': 'FormNumber',
    'Lease No': 'LeaseNumber',
    'Gas ID': 'Gasident'
}
h82009r = h82009.rename(column_names)
h82010r = h82010.rename(column_names)
h82011r = h82011.rename(column_names)
h82012r = h82012.rename(column_names)
h82013r = h82013.rename(column_names)
h82014r = h82014.rename(column_names)
h82015r = h82015.rename(column_names)
h82016r = h82016.rename(column_names)
h82017r = h82017.rename(column_names)

# combining into a single table
spills = agate.Table.merge([
        h82009r,
        h82010r,
        h82011r,
        h82012r,
        h82013r,
        h82014r,
        h82015r,
        h82016r,
        h82017r
    ])
# add GrossLossGal column converts barrels to gallons
spills = spills.compute([
    ('GrossLossGal', agate.Formula(agate.Number(), lambda r: r['GrossLoss'] * 42)),
    ('NetLossGal', agate.Formula(agate.Number(), lambda r: r['NetLoss'] * 42)),
    ('RecoverdGal', agate.Formula(agate.Number(), lambda r: r['Recovered'] * 42))
])

In [58]:
# send to csv to use elsewhere
spills.to_csv('exports/spills.csv')

In [59]:
# look at column names
print(spills)

| column           | data_type |
| ---------------- | --------- |
| MasterYear       | Text      |
| FormNumber       | Number    |
| District         | Text      |
| OperatorNumber   | Text      |
| DateofLoss       | Date      |
| Type_Company_Cd  | Text      |
| County           | Text      |
| LeaseNumber      | Number    |
| Gasident         | Number    |
| CommNbr          | Number    |
| TypeLiquid       | Text      |
| GrossLoss        | Number    |
| Recovered        | Number    |
| NetLoss          | Number    |
| Facility         | Text      |
| CauseofLoss      | Text      |
| Remedy           | Text      |
| Corrected_Report | Boolean   |
| Water            | Text      |
| WaterName        | Text      |
| Location         | Text      |
| Remarks          | Text      |
| LeaseName        | Text      |
| FieldNumber      | Text      |
| Into_Water       | Boolean   |
| Unique_ID        | Number    |
| GrossLossGal     | Number    |
| NetLossGal       | Number    |
| Recoverd

## Overall spills by GrossLoss
These are the top spills by the most lost.

In [60]:
# checking number of records for spills
print('Total number of records:\n{}'.format(
    spills.aggregate(agate.Count())
))


Total number of records:
8903


In [61]:
# orders the spills table
spills_ordered = spills.order_by(lambda row: row['GrossLoss'], reverse=True)
# prints just to show the top in the list
spills_ordered.select(['DateofLoss', 'County', 'GrossLoss']).limit(20).print_table()

| DateofLoss | County       | GrossLoss |
| ---------- | ------------ | --------- |
| 2012-10-01 | HARRISON     |    31,000 |
| 2010-02-09 | SAN PATRICIO |    25,000 |
| 2010-08-08 | BRAZOS       |    15,000 |
| 2010-10-11 | HOCKLEY      |    10,000 |
| 2016-09-10 | NOLAN        |     8,600 |
| 2011-01-27 | GRIMES       |     6,911 |
| 2014-09-15 | HOUSTON      |     5,700 |
| 2013-12-17 | ECTOR        |     5,000 |
| 2017-01-30 | COLLIN       |     5,000 |
| 2015-05-20 | STEPHENS     |     3,900 |
| 2013-05-20 | STARR        |     3,500 |
| 2009-06-17 | SCURRY       |     3,416 |
| 2015-06-20 | FREESTONE    |     3,300 |
| 2015-04-07 | TERRY        |     3,300 |
| 2016-04-06 | LOVING       |     3,200 |
| 2012-06-28 | DIMMIT       |     3,000 |
| 2011-04-01 | HOWARD       |     2,583 |
| 2012-01-30 | GAINES       |     2,572 |
| 2015-05-11 | STEPHENS     |     2,500 |
| 2009-01-29 | NUECES       |     2,400 |


In [62]:
## would like to count number of spills greater than 2,084 gallons.
spills_bigger = spills_ordered.where(lambda row: row['GrossLoss'] > bastrop_barrels)

print('Looking at all spills by GrossLoss, the Bastrop spill ranks {} in list of worst spills.'.format(
        spills_bigger.aggregate(agate.Count()) + 1
    ))

Looking at all spills by GrossLoss, the Bastrop spill ranks 24 in list of worst spills.


## Overall spills by NetLoss
Reordering all spills by NetLoss to account for barrels recovered. I'm not sure if there was oil recovered from the Bastrop spill, though.


In [63]:
spills_netloss = spills.order_by(lambda row: row['NetLoss'], reverse=True)
spills_netloss.select(['DateofLoss', 'County', 'NetLoss']).print_table()

| DateofLoss | County    | NetLoss |
| ---------- | --------- | ------- |
| 2012-10-01 | HARRISON  |  31,000 |
| 2016-09-10 | NOLAN     |   6,725 |
| 2014-09-15 | HOUSTON   |   5,698 |
| 2013-12-17 | ECTOR     |   5,000 |
| 2013-05-20 | STARR     |   3,500 |
| 2011-01-27 | GRIMES    |   3,304 |
| 2012-06-28 | DIMMIT    |   3,000 |
| 2011-04-01 | HOWARD    |   2,583 |
| 2015-06-15 | KENT      |   2,063 |
| 2013-10-28 | REAGAN    |   2,000 |
| 2009-06-17 | SCURRY    |   1,835 |
| 2014-09-27 | CULBERSON |   1,700 |
| 2010-08-09 | REAGAN    |   1,600 |
| 2012-01-30 | GAINES    |   1,472 |
| 2015-02-01 | FORT BEND |   1,452 |
| 2013-12-14 | ECTOR     |   1,430 |
| 2013-06-30 | UPTON     |   1,352 |
| 2015-11-23 | ATASCOSA  |   1,213 |
| 2015-01-07 | WEBB      |   1,130 |
| 2014-11-04 | KLEBERG   |   1,100 |
|        ... | ...       |     ... |


## Spills by 'Facility'

In [64]:
spills_group_facility = spills.group_by('Facility')
spills_facility_table = spills_group_facility.aggregate([
#         ('spill_cnt', agate.Count()),
#         ('spill_sum', agate.Sum('GrossLoss')),
        ('spill_sum_gal', agate.Sum('GrossLossGal')),
#         ('spill_avg', agate.Mean('GrossLoss')),
#         ('spill_med', agate.Median('GrossLoss')),
#         ('rec_sum', agate.Sum('Recovered')),
        ('rec_sum_gal', agate.Sum('RecoverdGal')),
#         ('net_sum', agate.Sum('NetLoss')),
        ('net_sum_gal', agate.Sum('NetLossGal'))
        
    ]
)
spills_facility_table.print_table()

| Facility             | spill_sum_gal | rec_sum_gal | net_sum_gal |
| -------------------- | ------------- | ----------- | ----------- |
| Tank Battery         |    24,005,436 |  12,916,512 |  11,088,924 |
| Flow Line            |     2,620,674 |   1,384,236 |   1,236,438 |
| Other                |     1,250,088 |     867,216 |     382,872 |
| Valve                |       846,846 |     591,906 |     254,940 |
| Pipe Line            |     4,036,872 |   2,505,552 |   1,531,320 |
| Pump Station or T... |       450,618 |     363,846 |      86,772 |
| Stuffing Box         |       121,842 |      81,942 |      39,900 |
| Separator-Gun Bar... |       276,990 |     176,694 |     100,296 |
| Vehicle (Truck-Ba... |        46,410 |      21,672 |      24,738 |


## Pipeline spills

In [65]:
# filter to pipeline spills
spills_pipelines = spills_ordered.where(lambda row: row['Facility'] == 'Pipe Line')

pipeline_spill_count = spills_pipelines.aggregate(agate.Count())

# count result
print('Number in spills_pipline:\n{}'.format(
  pipeline_spill_count
    ))



Number in spills_pipline:
673


### Pipeline spills over time

In [73]:
spills_pipeline_group_year = spills_pipelines.group_by('MasterYear')
spills_pipeline_year_table = spills_pipeline_group_year.aggregate([
        ('spill_sum_gal', agate.Sum('GrossLossGal')),
        ('rec_sum_gal', agate.Sum('RecoverdGal')),
        ('net_sum_gal', agate.Sum('NetLossGal')),
        ('spill_cnt', agate.Count()),
        ('spill_sum', agate.Sum('GrossLoss')),
#         ('spill_avg', agate.Mean('GrossLoss')),
#         ('spill_med', agate.Median('GrossLoss')),
        ('rec_sum', agate.Sum('Recovered')),
        ('net_sum', agate.Sum('NetLoss'))
    ]
)
# send to file for later
spills_pipeline_year_table.order_by('MasterYear').to_csv('exports/spills_pipeline_year_table.csv')
# print table
spills_pipeline_year_table.order_by('MasterYear').print_table()

| MasterYear | spill_sum_gal | rec_sum_gal | net_sum_gal | spill_cnt | spill_sum | ... |
| ---------- | ------------- | ----------- | ----------- | --------- | --------- | --- |
| 2009       |       253,512 |     154,350 |      99,162 |        66 |     6,036 | ... |
| 2010       |       897,204 |     744,786 |     152,418 |        77 |    21,362 | ... |
| 2011       |       580,566 |     317,688 |     262,878 |        70 |    13,823 | ... |
| 2012       |       467,754 |     302,904 |     164,850 |       109 |    11,137 | ... |
| 2013       |       490,308 |     294,672 |     195,636 |       107 |    11,674 | ... |
| 2014       |       335,244 |     196,350 |     138,894 |       114 |     7,982 | ... |
| 2015       |       298,452 |     142,086 |     156,366 |        80 |     7,106 | ... |
| 2016       |       502,488 |     169,638 |     332,850 |        46 |    11,964 | ... |
| 2017       |       211,344 |     183,078 |      28,266 |         4 |     5,032 | ... |


### Pipeline spills by year

In [74]:
spills_pipeline_year_table.order_by('MasterYear').print_bars('MasterYear', 'spill_cnt', width=60)

MasterYear spill_cnt
2009              66 ▓░░░░░░░░░░░░░                         
2010              77 ▓░░░░░░░░░░░░░░░                       
2011              70 ▓░░░░░░░░░░░░░                         
2012             109 ▓░░░░░░░░░░░░░░░░░░░░░                 
2013             107 ▓░░░░░░░░░░░░░░░░░░░░                  
2014             114 ▓░░░░░░░░░░░░░░░░░░░░░░                
2015              80 ▓░░░░░░░░░░░░░░░                       
2016              46 ▓░░░░░░░░░                             
2017               4 ▓░                                     
                     +---------+--------+--------+---------+
                     0        50       100      150      200


### Pipeline barrels spills by year

In [75]:
print('Barrels spilled from pipelines per year:\n')
spills_pipeline_year_table.order_by('MasterYear').print_bars('MasterYear', 'spill_sum', width=60)

Barrels spilled from pipelines per year:

MasterYear spill_sum
2009           6,036 ▓░░░░░░░░                              
2010          21,362 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░           
2011          13,823 ▓░░░░░░░░░░░░░░░░░░                    
2012          11,137 ▓░░░░░░░░░░░░░░                        
2013          11,674 ▓░░░░░░░░░░░░░░░                       
2014           7,982 ▓░░░░░░░░░░                            
2015           7,106 ▓░░░░░░░░░                             
2016          11,964 ▓░░░░░░░░░░░░░░░                       
2017           5,032 ▓░░░░░░                                
                     +---------+--------+--------+---------+
                     0       7,500   15,000   22,500  30,000


In [76]:
print('Gallons spilled from pipelines per year:\n')
spills_pipeline_year_table.order_by('MasterYear').print_bars('MasterYear', 'spill_sum_gal', width=60)


Gallons spilled from pipelines per year:

MasterYear spill_sum_gal
2009             253,512 ▓░░░░░░░░░░                        
2010             897,204 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
2011             580,566 ▓░░░░░░░░░░░░░░░░░░░░░░            
2012             467,754 ▓░░░░░░░░░░░░░░░░░░                
2013             490,308 ▓░░░░░░░░░░░░░░░░░░░               
2014             335,244 ▓░░░░░░░░░░░░░                     
2015             298,452 ▓░░░░░░░░░░░                       
2016             502,488 ▓░░░░░░░░░░░░░░░░░░░               
2017             211,344 ▓░░░░░░░░                          
                         +-------+--------+----------------+
                         0    225,000  450,000       900,000


### Top pipeline spills

In [ ]:
print('Top pipeline spills in gross loss barrels:\n')

# selecting and printing top records
spills_pipelines.limit(10).select(['DateofLoss', 'County', 'GrossLoss']).print_table()

## would like to count number of spills greater than 2,084 gallons.
pipeline_spills_bigger = spills_pipelines.where(lambda row: row['GrossLoss'] > bastrop_barrels)

print('\nLooking at all PIPELINE spills by GrossLoss, the Bastrop\
spill ranks {} out of {} in list of worst spills.'.format(
        pipeline_spills_bigger.aggregate(agate.Count()) + 1,
        pipeline_spill_count
    ))

### Pipelines severity of spills

In [ ]:
# histogram bin by GrossLoss
spills_pipeline_bins = spills_pipelines.bins('GrossLossGal', 10, 0, 10000)
spills_pipeline_bins.print_bars('GrossLossGal', 'Count', width=60)
# export bins for graphic
spills_pipeline_bins.to_csv('exports/spills_pipeline_bins.csv')

## Looking by county
### Bastrop spills

In [ ]:
# list of columns to print when looking at county results.
county_columns = [
    'County',
    'DateofLoss',
    'OperatorNumber',
    'Facility',
    'GrossLoss',
    'Unique_ID'
]

bastrop = spills_ordered.where(lambda row: row['County'] == 'BASTROP')

bastrop.select(county_columns).print_table(max_column_width=None)

### Travis spills

In [ ]:
travis = spills_ordered.where(lambda row: row['County'] == 'TRAVIS')

travis.select(county_columns).print_table(max_column_width=None)

### Galveston spills

In [ ]:
galveston = spills_ordered.where(lambda row: row['County'] == 'GALVESTON')

galveston.select(county_columns).order_by('DateofLoss').print_table(max_column_width=None)

### Harris spills

In [ ]:
harris = spills_ordered.where(lambda row: row['County'] == 'HARRIS')
harris.select(county_columns).order_by('DateofLoss').print_table(max_column_width=None)

### Feb 2011 spills

In [ ]:
# spills_2011 = spills.where(lambda row: row['DateofLoss'].year == 2011)
# spills_2011_02 = spills_2011.where(lambda row: row['DateofLoss'].month == 2)
# spills_2011_02.order_by('DateofLoss').select(county_columns).print_table(max_rows=None)

## Spills per year

In [ ]:
spills_group_year = spills.group_by('MasterYear')
spills_year_table = spills_group_year.aggregate([
        ('spill_cnt', agate.Count()),
        ('spill_sum', agate.Sum('GrossLoss')),
        ('spill_avg', agate.Mean('GrossLoss')),
        ('spill_med', agate.Median('GrossLoss')),
        ('spill_sum_gal', agate.Sum('GrossLossGal'))
    ]
)
spills_year_table.print_table()

### Number of spills

In [ ]:
spills_year_table.print_bars('MasterYear', 'spill_cnt', width=60)

### Barrels per year spilled

In [ ]:
spills_year_table.print_bars('MasterYear', 'spill_sum', width=60)

## Searching for the company
Looking for Magellan Midstream Partners or anything like them any any columns

In [ ]:
# this function allows me to imput a column name
# and get distinct matchinhg results
def search_column(column_name):
    result = spills.where(lambda row: re.search(
        'MAGELLAN|MIDSTREAM', str(row[column_name])
    ))
    
    print('Searching the \'{}\' column for MAGELLAN or MIDSTREAM \
    and found these distinct names. (If blank, there are none):'.format(
        column_name
        ))
    
    return result.columns[column_name].values_distinct()

In [ ]:
search_column('OperatorNumber')

In [ ]:
search_column('LeaseName')

In [ ]:
search_column('FieldNumber')

In [ ]:
search_column('Location')

In [ ]:
search_column('Remarks')

### Bulletproofing

Various looks to make sure all is good

In [ ]:
# check to make sure UniqueID is unique
pivot_UniqueID = spills.pivot('Unique_ID')
pivot_UniqueID.limit(5).print_table()

In [ ]:
# checks this 'Corrected_Report' column, which is only in some files, to see if any are true.
# None are
spills_corrected_pivot = spills.pivot('Corrected_Report', 'MasterYear')
spills_corrected_pivot.print_table(max_columns=None)